<a href="https://colab.research.google.com/gist/hardingnj/ffa0ee973fa59f3c11d0b6e3e7f64dfd/getting_started_google_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ag1000G phase 3 - cloud data access and analysis guide

## Overview

As part of phase 3 of the AG1000G project, we have publicly released genome-wide SNP data for 3483 Anopheline mosquitoes. Samples have been collected from 19 different African countries at various time points. Three species are represented in the dataset, Anopheles gambiae, coluzzii and arabiensis. For more details of the project please see  the project page: malariagen.net/

Individual SNP calls are provided unfiltered. This is because sites considered reliable in one species may not be in another. We provide 3 different site filter masks for analyses including different species. For more details see the methods produced as part of the release.

In this dataset we expect to see signals of selection and incipient speciation, and therefore represents a potentially rich resource for population geneticists and vector biologists.

## Introduction

This notebook provides some examples of accessing data from the Ag1000G phase 3 SNP release on Google Cloud Services (GCS). It illustrates how to read data directly from Google Cloud, without having to first download any data locally. This notebook can be run from any computer, but will work best when run from a compute node within Google Cloud, because it will be physically closer to the data and so data transfer is faster. For example, this notebook can be run via [Google Colab](https://colab.research.google.com/) which is an interactive computing service running in Google Cloud.

If you find Google Colab too slow to run specific analyses, there may be a case to use MalariaGEN's DataLab service, which also runs inside Google Cloud but offers more compute power. Please email data@malariagen.net if you'd like to discuss this. 

Please address any queries about this notebook to nicholas.harding@bdi.ox.ac.uk.
Or raise an issue [here](https://github.com/malariagen/vector-public-data/issues).

## About this guide

This guide is written as a Jupyter notebook, and describes the analysis of single nucleotide polymorphisms and associated sample metadata _only_. If you wish to perform specific analyses of sequence read alignments you will need to download data locally, see [here](download_guide_path_tbc).

## Data hosting

All data required for this notebook is hosted on Google Cloud Storage. Additional files, such as alignments (`.bam`) and SNP calls in `VCF` are available via the European Nucleotide Archive and the European Variation Archive respectively.

Sample metadata and SNP calls in Zarr format are hosted on Google Cloud Storage (GCS) in the vo_agam_release bucket, which is a multi-region bucket located in the United States. All data hosted in GCS are publicly accessible and do not require any authentication to access. This guide provides examples of using these data directly from a 
Sample sets

## Data structure

Data in this release are organised into 28 sample sets. Each of these sample sets corresponds to a set of mosquito specimens contributed by a collaborating study. Depending on your objectives, you may want to download data from only specific sample sets, or all sample sets. For convenience there is a tab-delimited manifest file listing all sample sets in the release. Here is a direct download link for the sample set manifest:

The sample set identifiers all start with "AG1000G-" followed by the two-letter code of the country from which samples were collected (e.g., "AO" is Angola). Where there are multiple sample sets from the same country, these have been given alphabetical suffixes, e.g., "AG1000G-BF-A", "AG1000G-BF-B" and "AG1000G-BF-C" are three sample sets from Burkina Faso.

These country code suffixes are just a convenience to help remember which sample sets contain which data, please see the sample metadata for more precise location information. Please note also that sample set AG1000G-GN-B contains samples from both Guinea and Mali.

More information on sample sets is provided below. 

In [ ]:
# Install additional packages.
!pip install -q intake zarr gcsfs fsspec scikit-allel

In [2]:
# Import packages.
import intake
import allel
import pandas as pd
import numpy as np
import dask.array as da
from dask.diagnostics import ProgressBar as progress
import matplotlib.pyplot as plt
%matplotlib inline

## Data catalog

For convenience, all data from Ag1000G phase 3 can be accessed via an [intake](https://intake.readthedocs.io/en/latest/index.html) data catalog. The code below illustrates how to load the catalog.

In [3]:
# Open the Ag1000G phase 3 data catalog.
cat = intake.open_catalog("https://malariagen.github.io/intake/gcs.yml")

## Sample sets

All of the data in Ag1000G phase 3 is organised into sample sets. There are 28 sample sets in total. In general, each sample set contains samples from a single country and contributor. 

To see which sample sets are available, load a [pandas](https://pandas.pydata.org/) dataframe that lists sample sets:

In [4]:
# Read in the list of available sample sets.
df_sample_sets = cat.ag3.sample_sets.read()
df_sample_sets

/home/njh/dev/vector-ops/binder/deps/conda/envs/vector-ops-v2.5.1/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


,sample_set,sample_count
0,AG1000G-AO,81
1,AG1000G-BF-A,181
2,AG1000G-BF-B,102
3,AG1000G-BF-C,13
4,AG1000G-CD,76
5,AG1000G-CF,73
6,AG1000G-CI,80
7,AG1000G-CM-A,303
8,AG1000G-CM-B,97
9,AG1000G-CM-C,44


## Sample metadata

Sample metadata such as the collection location and year are available for all samples. These are organised by sample set.

Importantly, the order of rows in sample metadata correspond to the columns in the SNP genotype data. So if you wish to exclude the _i_ th sample from your analysis, the _i_ th column of the SNP data array should be excluded.

E.g., load sample metadata for the AG1000G-TZ sample set:

In [5]:
df_samples = cat.ag3.samples(sample_set="AG1000G-TZ").read()
df_samples.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call
0,BL0046-C,Plate_C_H6,Bilali Kabula,Tanzania,Muleba,2015,6,-1.962,31.651,F
1,BL0047-C,Plate_F_D4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F
2,BL0048-C,Plate_F_E4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F
3,BL0049-C,Plate_F_F4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F
4,BL0050-C,Plate_F_G4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F


In [6]:
df_samples.groupby(["country", "location", "year"]).size()

country   location  year
Tanzania  Moshi     2012     40
          Muheza    2013     43
          Muleba    2015    170
          Tarime    2012     47
dtype: int64

## Species calls

Samples within a sample set may belong to different species. If the user wishes to restrict their analysis to specific species, they should filter along columns. The final example provided in this notebook shows an example restricting an analysis to _A. coluzzii_.

For convenience, we have made a species call for all samples in Ag1000G phase 3, using the genomic data. We have used two different methods for species calling:

1. Ancestry informative markers (AIMs)
2.  Principal components analysis (PCA)

Note that the results of these two different methods generally agree, although there are some populations where results are different, particularly in Guinea-Bissau and The Gambia. If you have any questions about how to interpret these species calls, please get in touch.

E.g., read the AIM species calls for sample set AG1000G-TZ:

In [7]:
df_species_aim = cat.ag3.species_calls_20200422_aim(sample_set="AG1000G-TZ").read()
df_species_aim.head()

,sample_id,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii
0,BL0046-C,0.125,0.005,gamb_colu,intermediate
1,BL0047-C,0.459,0.745,arabiensis,NaN
2,BL0048-C,0.455,0.742,arabiensis,NaN
3,BL0049-C,0.455,0.744,arabiensis,NaN
4,BL0050-C,0.451,0.735,arabiensis,NaN


E.g., read the PCA species calls for sample set AG1000G-TZ:

In [8]:
df_species_pca = cat.ag3.species_calls_20200422_pca(sample_set="AG1000G-TZ").read()
df_species_pca.head()

,sample_id,PC1,PC2,species_gambcolu_arabiensis,species_gambiae_coluzzii
0,BL0046-C,-27.700,-18.597,gamb_colu,gambiae
1,BL0047-C,202.946,2.731,arabiensis,NaN
2,BL0048-C,198.683,4.455,arabiensis,NaN
3,BL0049-C,202.344,4.364,arabiensis,NaN
4,BL0050-C,197.466,3.327,arabiensis,NaN


In [9]:
df_samples.shape

(300, 10)

In [10]:
df_species_pca.shape

(300, 5)

In [11]:
# merge the metadata with the species calls.
df_samples.merge(df_species_aim, on="sample_id").groupby(
    ["country", "location", "year", "species_gambcolu_arabiensis"]).size()

country   location  year  species_gambcolu_arabiensis
Tanzania  Moshi     2012  arabiensis                      40
          Muheza    2013  arabiensis                       1
                          gamb_colu                       42
          Muleba    2015  arabiensis                     137
                          gamb_colu                       33
          Tarime    2012  arabiensis                      47
dtype: int64

## SNP sites and alleles

We have called SNP genotypes in all samples at all positions in the genome where the reference allele is not "N". Data on this set of genomic positions, and the coding of reference and alternate alleles, is available as a [zarr](https://zarr.readthedocs.io/) hierarchy, and can be opened directly from the data catalog: 

In [12]:
callset_sites = cat.ag3.snp_sites.to_zarr()
callset_sites

<zarr.hierarchy.Group '/' read-only>

The data are grouped by chromosome arm:

In [13]:
list(callset_sites)

['2L', '2R', '3L', '3R', 'Mt', 'UNKN', 'X', 'Y_unplaced']

For each chromosome arm, the "variants/POS" array gives the genomic positions, the "variants/REF" array gives the reference alleles, and the "variants/ALT" array gives the alternate alleles. These data can be loaded into [numpy](https://numpy.org/) arrays by slicing.

E.g., load genomic positions for the first 10 SNPs on chromosome arm 3R:

In [14]:
callset_sites["3R/variants/POS"][:10]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int32)

E.g., load reference alleles for the first 10 SNPs on chromosome arm 3R:

In [15]:
callset_sites["3R/variants/REF"][:10]

array([b'C', b'C', b'T', b'C', b'T', b'A', b'C', b'G', b'T', b'T'],
      dtype='|S1')

E.g., load alternate alleles for the first 10 SNPs on chromosome arm 3R:

In [16]:
callset_sites["3R/variants/ALT"][:10]

array([[b'A', b'T', b'G'],
       [b'A', b'T', b'G'],
       [b'A', b'C', b'G'],
       [b'A', b'T', b'G'],
       [b'A', b'C', b'G'],
       [b'C', b'T', b'G'],
       [b'A', b'T', b'G'],
       [b'A', b'C', b'T'],
       [b'A', b'C', b'G'],
       [b'A', b'C', b'G']], dtype='|S1')

## Site filters

SNP calling is not always reliable, and we have created some site filters to allow excluding low quality SNPs. 

Because different species may have different genome accessibility issues, we have created three separate site filters:

* The "gamb_colu" filter is design for working only with samples that **are not** An. arabiensis. 
* The "arab" filter is designed for when only working with samples that **are** An. arabiensis. 
* The "gamb_colu_arab" filter is suitable for when analysing samples of any species together. 

The site filters data are available as zarr hierarchies. 

E.g., open the "gamb_colu" site filters:

In [17]:
callset_filters_gamb_colu = cat.ag3.site_filters_dt_20200416_gamb_colu.to_zarr()

E.g., open the "arab" site filters:

In [18]:
callset_filters_arab = cat.ag3.site_filters_dt_20200416_arab.to_zarr()

E.g., open the "gamb_colu_arab" site filters:

In [19]:
callset_filters_gamb_colu_arab = cat.ag3.site_filters_dt_20200416_gamb_colu_arab.to_zarr()

Each set of site filters provides a "filter_pass" Boolean mask for each chromosome arm, where True indicates that the site passed the filter and is accessible to high quality SNP calling. 

E.g., load the mask for the first 10 SNPs on chromosome arm 3R:

In [20]:
callset_filters_gamb_colu["3R/variants/filter_pass"][:10]

array([False, False, False, False, False, False, False, False, False,
       False])

## SNP genotypes

SNP genotypes for individual samples are available as zarr hierarchies, one for each sample set. 

For each sample set, data are grouped by chromosome arm. The "calldata/GT" array provides the actual genotypes, the "calldata/GQ" array provides genotype qualities, the "calldata/MQ" provides mapping qualities, and the "calldata/AD" provides allele depths.

E.g., here is an overview of the arrays available for the AG1000G-TZ sample set:

In [21]:
callset_genotypes = cat.ag3.snp_genotypes(sample_set="AG1000G-TZ").to_zarr()
print(callset_genotypes.tree())

/
 ├── 2L
 │   └── calldata
 │       ├── AD (48525747, 300, 4) int16
 │       ├── GQ (48525747, 300) int16
 │       ├── GT (48525747, 300, 2) int8
 │       └── MQ (48525747, 300) int16
 ├── 2R
 │   └── calldata
 │       ├── AD (60132453, 300, 4) int16
 │       ├── GQ (60132453, 300) int16
 │       ├── GT (60132453, 300, 2) int8
 │       └── MQ (60132453, 300) int16
 ├── 3L
 │   └── calldata
 │       ├── AD (40758473, 300, 4) int16
 │       ├── GQ (40758473, 300) int16
 │       ├── GT (40758473, 300, 2) int8
 │       └── MQ (40758473, 300) int16
 ├── 3R
 │   └── calldata
 │       ├── AD (52226568, 300, 4) int16
 │       ├── GQ (52226568, 300) int16
 │       ├── GT (52226568, 300, 2) int8
 │       └── MQ (52226568, 300) int16
 ├── X
 │   └── calldata
 │       ├── AD (23385349, 300, 4) int16
 │       ├── GQ (23385349, 300) int16
 │       ├── GT (23385349, 300, 2) int8
 │       └── MQ (23385349, 300) int16
 └── samples (300,) |S24


If you want to work with the genotype calls, you may find it convenient to use [scikit-allel](http://scikit-allel.readthedocs.org/). E.g., this code sets up a genotype array:

In [22]:
# use the scikit-allel Genotype array class
gt = allel.GenotypeDaskArray(callset_genotypes["3R/calldata/GT"])
gt

<GenotypeDaskArray shape=(52226568, 300, 2) dtype=int8>

The code below subsets the data to sites that pass the gamb_colu site filter, then performs an allele count across all pass sites in the chromosome arm. It takes a little while because it is scanning data for millions of SNPs:

In [23]:
loc_pass = callset_filters_gamb_colu["3R/variants/filter_pass"][:]
gt_pass = gt[loc_pass]
with progress():
    ac_pass = gt_pass.count_alleles(max_allele=3).compute()
ac_pass

[########################################] | 100% Completed | 11min 44.4s


<AlleleCountsArray shape=(37199402, 4) dtype=int32>
600   0   0   0
598   0   0   0
600   0   0   0
...
600   0   0   0
600   0   0   0
600   0   0   0

# Concatenating data from multiple sample sets

Often you may wish to work with multiple sample sets simultaneously, in which 
case data needs to concatenated.

Concatenating can be done directly. E.g., concatenate sample metadata for two sample sets:



In [24]:
sample_sets = ["AG1000G-TZ", "AG1000G-UG"]
df_samples = pd.concat(
    [cat.ag3.samples(sample_set=s).read() for s in sample_sets], 
    axis=0)
df_samples.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call
0,BL0046-C,Plate_C_H6,Bilali Kabula,Tanzania,Muleba,2015,6,-1.962,31.651,F
1,BL0047-C,Plate_F_D4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F
2,BL0048-C,Plate_F_E4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F
3,BL0049-C,Plate_F_F4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F
4,BL0050-C,Plate_F_G4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F


In [25]:
df_samples.groupby(["country", "location", "year"]).size()

country   location   year
Tanzania  Moshi      2012     40
          Muheza     2013     43
          Muleba     2015    170
          Tarime     2012     47
Uganda    Kihihi     2012     96
          Nagongera  2012    194
dtype: int64

In [26]:
len(df_samples)

590

SNP genotypes for multiple sample sets can be concatenated using [dask](https://docs.dask.org/en/latest/array.html). E.g.:

In [27]:
callsets = [cat.ag3.snp_genotypes(sample_set=s).to_zarr() for s in sample_sets]
gt_arrays = [da.from_array(callset["3R/calldata/GT"]) for callset in callsets]

# note that arrays are concatenated across axis 1
gt = allel.GenotypeDaskArray(da.concatenate(gt_arrays, axis=1))
gt

<GenotypeDaskArray shape=(52226568, 590, 2) dtype=int8>

## Convenience functions for accessing data

Below are some functions for accessing data more easily from the catalog.

In [28]:
def load_mask(seq_id, mask, filters_analysis="dt_20200416"):
    """Load a site filter mask.
    
    Parameters
    ----------
    seq_id : str
        Chromosome arm.
    mask : {"gamb_colu", "arab", "gamb_colu_arab"}
        Mask species combination.
    filters_analysis : str
        Filtering model.

    Returns
    -------
    filter_pass: numpy array
        Boolean array where True means pass.

    """ 
    
    callset_filters = cat.ag3[f"site_filters_{filters_analysis}_{mask}"].to_zarr()
    filter_pass = callset_filters[seq_id]["variants/filter_pass"][:]

    return filter_pass

In [29]:
load_mask("3R", "gamb_colu")

array([False, False, False, ..., False, False, False])

In [30]:
def load_variants_array(seq_id, field, mask=None, filters_analysis="dt_20200416"):
    """Load sites data.
    
    Parameters
    ----------
    seq_id : str
        Chromosome arm.
    field : {"REF", "ALT", "POS"}
        Array to load.
    mask : {"gamb_colu", "arab", "gamb_colu_arab"}
        Mask species combination.
    filters_analysis : str
        Filtering model.

    Returns
    -------
    arr : numpy array

    """
    
    callset_sites = cat.ag3.snp_sites.to_zarr()
    arr = da.from_zarr(callset_sites[seq_id]["variants"][field])

    if mask is not None:
        filter_pass = load_mask(seq_id, mask, filters_analysis)
        arr = da.compress(filter_pass, arr, axis=0)

    return arr.compute()

In [31]:
pos = load_variants_array("3R", "POS", mask="gamb_colu")
pos

array([     180,      185,      236, ..., 53196502, 53196504, 53196522],
      dtype=int32)

In [32]:
def load_calldata_array(seq_id, sample_set, field, mask=None, filters_analysis="dt_20200416"):
    """Load SNP genotype calldata for a given sample set.

    Parameters
    ----------
    seq_id : str
        Chromosome arm.
    sample_set : str
        Sample set.
    field : {"GT", "GQ", "AD", "MQ"}
        Array to load.
    mask : {"gamb_colu", "arab", "gamb_colu_arab"}
        Mask species combination.
    filters_analysis : str
        Filtering model.

    Returns
    -------
    arr : dask.Array
    
    """
    
    if isinstance(sample_set, str):
        # load data for a single sample set
        callset_genotypes = cat.ag3.snp_genotypes(sample_set=sample_set).to_zarr()
        arr = da.from_zarr(callset_genotypes[f"{seq_id}/calldata/{field}"])
                
    elif isinstance(sample_set, (list, tuple)):
        # load data for multiple sample sets
        arr = da.concatenate(
            [load_calldata_array(
                seq_id=seq_id, 
                sample_set=x, 
                field=field, 
                mask=None) for x in sample_set], 
            axis=1)
    
    else:
        raise TypeError("Type of `sample_set` must be string or list of strings")
        
    if mask is not None:
    
        filter_pass = load_mask(seq_id, mask, filters_analysis)
        arr = da.compress(filter_pass, arr, axis=0)

    if field == "GT":
        arr = allel.GenotypeDaskArray(arr)
            
    return arr

In [33]:
gt = load_calldata_array("2L", sample_set=["AG1000G-TZ", "AG1000G-UG"], field="GT", mask="gamb_colu_arab")
gt

<GenotypeDaskArray shape=(32529983, 590, 2) dtype=int8>

In [34]:
def load_sample_metadata(sample_set, 
                         include_aim_species_calls=True, 
                         include_pca_species_calls=False, 
                         species_analysis="species_calls_20200422"):
    """Load sample metadata, optionally including species calls.

    Parameters
    ----------
    sample_set : str
        Sample set.
    include_aim_species_calls : bool
        If True, include AIM calls.
    include_pca_species_calls : bool
        If True, include PCA calls.
    species_analysis : str
        Species analysis.

    Returns
    -------
    df : pandas.DataFrame

    Notes
    -----
    If both AIMs and PCA are requested, species calls columns are appended with 
    "_aim" and "_pca" respectively.

    """

    if isinstance(sample_set, str):

        df = cat.ag3.samples(sample_set=sample_set).read()
        df["sample_set"] = sample_set

        if include_aim_species_calls:
            df_aim = cat.ag3[f"{species_analysis}_aim"](sample_set=sample_set).read()
            
        if include_pca_species_calls:
            df_pca = cat.ag3[f"{species_analysis}_pca"](sample_set=sample_set).read()

        df_species = None

        if include_aim_species_calls and include_pca_species_calls:
            df_species = df_aim.merge(df_pca, on="sample_id", lsuffix="_aim", rsuffix="_pca", sort=False)
            
        elif include_aim_species_calls:
            df_species = df_aim
            
        elif include_pca_species_calls:
            df_species = df_species

        if df_species is not None:
            df = df.merge(df_species, on="sample_id", sort=False)
    
        return df

    elif isinstance(sample_set, (list, tuple)):

        return pd.concat(
            [load_sample_metadata(
                sample_set=s, 
                include_aim_species_calls=include_aim_species_calls, 
                include_pca_species_calls=include_pca_species_calls, 
                species_analysis=species_analysis) 
             for s in sample_set],
            axis=0, sort=False)
    
    else:
        raise TypeError("Type of `sample_set` must be string or list of strings")
    

In [35]:
df_meta = load_sample_metadata(sample_set=["AG1000G-TZ", "AG1000G-UG"])
df_meta.head()

,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii
0,BL0046-C,Plate_C_H6,Bilali Kabula,Tanzania,Muleba,2015,6,-1.962,31.651,F,AG1000G-TZ,0.125,0.005,gamb_colu,intermediate
1,BL0047-C,Plate_F_D4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F,AG1000G-TZ,0.459,0.745,arabiensis,NaN
2,BL0048-C,Plate_F_E4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F,AG1000G-TZ,0.455,0.742,arabiensis,NaN
3,BL0049-C,Plate_F_F4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F,AG1000G-TZ,0.455,0.744,arabiensis,NaN
4,BL0050-C,Plate_F_G4,Bilali Kabula,Tanzania,Muleba,2015,3,-1.962,31.651,F,AG1000G-TZ,0.451,0.735,arabiensis,NaN


In [36]:
df_meta.shape

(590, 15)

In [37]:
df_meta.fillna("").groupby(by=["country", "location", "species_gambcolu_arabiensis", "species_gambiae_coluzzii"]).size()

country   location   species_gambcolu_arabiensis  species_gambiae_coluzzii
Tanzania  Moshi      arabiensis                                                40
          Muheza     arabiensis                                                 1
                     gamb_colu                    gambiae                      36
                                                  intermediate                  6
          Muleba     arabiensis                                               137
                     gamb_colu                    gambiae                      32
                                                  intermediate                  1
          Tarime     arabiensis                                                47
Uganda    Kihihi     arabiensis                                                 1
                     gamb_colu                    gambiae                      95
          Nagongera  arabiensis                                                81
                     ga

## Example analysis

- Plot nucleotide diversity in AG1000G-BF-A and AG1000G-BF-B over 2L

- Plot nucleotide diversity in the above sample sets, but only in _An. coluzzii_ according to AIM species call

In [38]:
sample_sets = ["AG1000G-BF-A", "AG1000G-BF-B"]
seq_id = "2L"
mask = "gamb_colu"

In [39]:
filter_pass = load_mask(seq_id=seq_id, mask=mask)

In [40]:
gt = load_calldata_array(seq_id=seq_id, field="GT", sample_set=sample_sets, mask=mask)
gt

<GenotypeDaskArray shape=(36005131, 283, 2) dtype=int8>

In [41]:
pos = load_variants_array(seq_id=seq_id, field="POS", mask=mask)
pos

array([    1215,     1216,     1231, ..., 49356403, 49356404, 49359100],
      dtype=int32)

In [42]:
with progress():
    ac = gt.count_alleles(max_allele=3).compute()
    pi, windows, nbases, counts = allel.windowed_diversity(
        pos, ac, is_accessible=filter_pass, size=100_000)

[########################################] | 100% Completed | 10min 47.9s


ValueError: Not all positions are covered by is_accessible.

In [ ]:
# mask windows with less than 10_000 accessible bases = 10%
pi[nbases < 10_000] = np.nan

In [ ]:
f, ax = plt.subplots(figsize=(8, 5))

ax.plot(windows.mean(axis=1), pi)
ax.grid(True)
ax.set_ylabel("nucleotide diversity")
ax.set_xlabel("genomic position (bp)");

In [ ]:
df_meta = load_sample_metadata(sample_set=sample_sets)

In [ ]:
# use the metadata to define a boolean that represents samples to keep
selected_samples = df_meta.species_gambiae_coluzzii == "coluzzii"
gt_col = gt.compress(selected_samples.values, axis=1)

In [ ]:
with progress():
    ac_col = gt_col.count_alleles(max_allele=3).compute()
    pi_col, windows_col, nbases_col, counts_col = allel.windowed_diversity(
        pos, ac_col, is_accessible=filter_pass, size=100_000)

In [ ]:
# mask windows with less than 10_000 accessible bases 10%
pi_col[nbases_col < 10_000] = np.nan

In [ ]:
f, ax = plt.subplots(figsize=(8, 5))

ax.plot(windows.mean(axis=1), pi_col, color="red")
ax.grid(True)
ax.set_ylabel("nucleotide diversity")
ax.set_xlabel("genomic position (bp)")